In [1]:
# Accessing Data from XLM Catalog via Siphon Libraries
from siphon.catalog import TDSCatalog
from siphon.ncss import NCSS

In [2]:
from datetime import datetime, timedelta

In [3]:
now = datetime.utcnow()

In [4]:
from siphon.ncss import open_var_browser

In [5]:
from siphon.ncss import thredds_model_dict

In [6]:
thredds_model_dict

{'RAP': {'CONUS_13km': 'RAP/CONUS_13km/RR_CONUS_13km',
  'CONUS_20km': 'RAP/CONUS_20km/RR_CONUS_20km',
  'CONUS_40km': 'RAP/CONUS_40km/RR_CONUS_40km'},
 'GFS': {'0p25_ana': 'GFS/Global_0p25deg_ana/GFS_Global_0p25deg_ana',
  '0p25': 'GFS/Global_0p25deg/GFS_Global_0p25deg',
  '0p5_ana': 'GFS/Global_0p5deg_ana/GFS_Global_0p5deg_ana',
  '0p5': 'GFS/Global_0p5deg/GFS_Global_0p5deg',
  'onedeg_ana': 'GFS/Global_onedeg_ana/GFS_Global_onedeg_ana',
  'onedeg': 'GFS/Global_onedeg/GFS_Global_onedeg',
  'Pac_20km': 'GFS/Pacific_20km/GFS_Pacific_20km',
  'PR_0p25': 'GFS/Puerto_Rico_0p25deg/GFS_Puerto_Rico_0p25deg',
  'CONUS_95km': 'GFS/CONUS_95km/GFS_CONUS_95km',
  'CONUS_80km': 'GFS/CONUS_80km/GFS_CONUS_80km',
  'CONUS_20km': 'GFS/CONUS_20km/GFS_CONUS_20km',
  'AK_20km': 'GFS/Alaska_20km/GFS_Alaska_20km'},
 'HRRR': {'CONUS_3km': 'HRRR/CONUS_3km/surface/HRRR_CONUS_3km',
  'CONUS_2p5km_ana': 'HRRR/CONUS_2p5km_ANA/HRRR_CONUS_2p5km_ana',
  'CONUS_2p5km': 'HRRR/CONUS_2p5km/HRRR_CONUS_2p5km'},
 'GEFS': 

In [7]:
list(thredds_model_dict.keys())

['RAP', 'GFS', 'HRRR', 'GEFS', 'NAM', 'NDFD', 'RTMA', 'NCEP Blend']

In [8]:
thredds_model_dict["GFS"]

{'0p25_ana': 'GFS/Global_0p25deg_ana/GFS_Global_0p25deg_ana',
 '0p25': 'GFS/Global_0p25deg/GFS_Global_0p25deg',
 '0p5_ana': 'GFS/Global_0p5deg_ana/GFS_Global_0p5deg_ana',
 '0p5': 'GFS/Global_0p5deg/GFS_Global_0p5deg',
 'onedeg_ana': 'GFS/Global_onedeg_ana/GFS_Global_onedeg_ana',
 'onedeg': 'GFS/Global_onedeg/GFS_Global_onedeg',
 'Pac_20km': 'GFS/Pacific_20km/GFS_Pacific_20km',
 'PR_0p25': 'GFS/Puerto_Rico_0p25deg/GFS_Puerto_Rico_0p25deg',
 'CONUS_95km': 'GFS/CONUS_95km/GFS_CONUS_95km',
 'CONUS_80km': 'GFS/CONUS_80km/GFS_CONUS_80km',
 'CONUS_20km': 'GFS/CONUS_20km/GFS_CONUS_20km',
 'AK_20km': 'GFS/Alaska_20km/GFS_Alaska_20km'}

In [9]:
now = datetime.utcnow()
# define time range you want the data for
start = now
print(start)
delta_t = 12
end = now + timedelta(hours=delta_t)

2020-10-07 22:23:33.812444


In [11]:
catalog_url = open_var_browser("GFS",'CONUS_20km',now,"0000")
catalog_url

'https://thredds.ucar.edu/thredds/catalog/grib/NCEP/GFS/CONUS_20km/GFS_CONUS_20km_20201007_0000.grib2/catalog.xml'

In [12]:
# Request the GFS data from the thredds server
#gfs_url = f"https://thredds.ucar.edu/thredds/catalog/grib/NCEP/GFS/Global_0p25deg/GFS_Global_0p25deg_{now.year}{now.month:02d}{now.day:02d}_0000.grib2/catalog.xml"

model = TDSCatalog(catalog_url)
    
dataset = list(model.datasets.values())[0]
#print(dataset.access_urls)
    
# Create NCSS object to access the NetcdfSubset
ncss = NCSS(dataset.access_urls['NetcdfSubset'])

In [13]:
query = ncss.query()
query.time_range(start, end)
Lat = query.lonlat_box(north=80, south=0, east=310, west=200)
query.accept('netcdf4')

time_start=2020-10-07T22%3A23%3A33.812444&time_end=2020-10-08T10%3A23%3A33.812444&west=200&east=310&south=0&north=80&accept=netcdf4

In [7]:
mslp_name = "" # What is the name in the catalog???????
# .
# .
# .
# check out the ncss.thredds_model_dict dictionary and ncss.open_var_browser() method
ncss.thredds_model_dict["GFS"]

{'0p25_ana': 'GFS/Global_0p25deg_ana/GFS_Global_0p25deg_ana',
 '0p25': 'GFS/Global_0p25deg/GFS_Global_0p25deg',
 '0p5_ana': 'GFS/Global_0p5deg_ana/GFS_Global_0p5deg_ana',
 '0p5': 'GFS/Global_0p5deg/GFS_Global_0p5deg',
 'onedeg_ana': 'GFS/Global_onedeg_ana/GFS_Global_onedeg_ana',
 'onedeg': 'GFS/Global_onedeg/GFS_Global_onedeg',
 'Pac_20km': 'GFS/Pacific_20km/GFS_Pacific_20km',
 'PR_0p25': 'GFS/Puerto_Rico_0p25deg/GFS_Puerto_Rico_0p25deg',
 'CONUS_95km': 'GFS/CONUS_95km/GFS_CONUS_95km',
 'CONUS_80km': 'GFS/CONUS_80km/GFS_CONUS_80km',
 'CONUS_20km': 'GFS/CONUS_20km/GFS_CONUS_20km',
 'AK_20km': 'GFS/Alaska_20km/GFS_Alaska_20km'}

In [15]:
current_catalog = open_var_browser("GFS","CONUS_20km",datetime.utcnow(),"0000",open_browser=True)

In [16]:
mslp_name = "MSLP_Eta_model_reduction_msl"
query.variables(mslp_name).add_lonlat(True)
# Request data for the variables you want to use
data = ncss.get_data(query)

var=MSLP_Eta_model_reduction_msl&time_start=2020-10-07T22%3A23%3A33.812444&time_end=2020-10-08T10%3A23%3A33.812444&west=200&east=310&south=0&north=80&accept=netcdf4&addLatLon=True

In [18]:
# don't have netcdf4 installed at the moment so data wont be pulled correctly
data

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    Originating_or_generating_Center: US National Weather Service, National Centres for Environmental Prediction (NCEP)
    Originating_or_generating_Subcenter: 0
    GRIB_table_version: 2,1
    Type_of_generating_process: Forecast
    Analysis_or_forecast_generating_process_identifier_defined_by_originating_centre: Analysis from GFS (Global Forecast System)
    Conventions: CF-1.6
    history: Read using CDM IOSP GribCollection v3
    featureType: GRID
    History: Translated to CF-1.0 Conventions by Netcdf-Java CDM (CFGridWriter2)
Original Dataset = /data/ldm/pub/native/grid/NCEP/GFS/CONUS_20km/GFS_CONUS_20km_20201007_0000.grib2.ncx3#LambertConformal_257X369-40p69N-100p4W; Translation Date = 2020-10-07T22:27:03.443Z
    geospatial_lat_min: 13.79481320192154
    geospatial_lat_max: 57.335950219792934
    geospatial_lon_min: -153.0345501621164
    geospatial_lon_max: -49.20529767978462
    dimensions(